# How to Generate Race & Ethnicity Predictions using ZRP
The purpose of this notebook is to illustrate how to use ZRP_Predict, a module that generates race & ethnicity predictions

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser, dirname
import pandas as pd
import sys
import os
import re
import warnings

In [3]:
warnings.filterwarnings(action='once')
home = expanduser('~')

src_path = '{}/zrp'.format(home)
sys.path.append(src_path)

In [4]:
from zrp.modeling.predict import ZRP_Predict
from zrp.prepare.prepare import ZRP_Prepare
from zrp.prepare.utils import load_file

/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/statsmodels/compat/pandas.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  version = LooseVersion(pd.__version__)
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Load sample data for prediction
Load processed list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [5]:
nj_mayors = load_file("../2022-nj-mayors-sample.csv")
nj_mayors.shape

(462, 9)

In [6]:
nj_mayors

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY
0,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2
1,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4
2,David,J.,Mclaughlin,125,Corlies Avenue,Allenhurst,NJ,07711-1049,5
3,Thomas,C.,Fritts,8,North Main Street,Allentown,NJ,08501-1607,6
4,P.,NaN,McCkelvey,49,South Greenwich Street,Alloway,NJ,08001-0425,7
...,...,...,...,...,...,...,...,...,...
457,William,NaN,Degroff,3943,Route,Chatsworth,NJ,08019,558
458,Joseph,NaN,Chukwueke,200,Cooper Avenue,Woodlynne,NJ,08107-2108,559
459,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560
460,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562


#### ZRP Prepare  
Predictions can only be generated from prepared data that is processed, Census GEOIDs (ie census tract), and has American Community Survey mapped to each unique record. To prepare the data we will use ZRP_Prepare

In [7]:
%%time
zest_race_predictor = ZRP_Prepare()
zest_race_predictor.fit(nj_mayors)
prepared = zest_race_predictor.transform(nj_mayors)

  0%|          | 0/462 [00:00<?, ?it/s][Parallel(n_jobs=49)]: Using backend ThreadingBackend with 49 concurrent workers.
[Parallel(n_jobs=49)]: Done 102 tasks      | elapsed:    0.0s
100%|██████████| 462/462 [00:00<00:00, 13029.73it/s]

Data is loaded
Directory already exists
   [Start] Validating input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing, this may impact the ability to return race approximations
   [Completed] Validating input data

   Formatting P1
   Formatting P2
   reduce whitespace

[Start] Preparing geo data

  The following states are included in the data: ['NJ']
   ... on state: NJ

   Data is loaded
   [Start] Processing geo data
/home/kam/zrp/zrp/prepare/../data/processed
      ...address cleaning



[Parallel(n_jobs=49)]: Done 352 tasks      | elapsed:    0.0s
[Parallel(n_jobs=49)]: Done 462 out of 462 | elapsed:    0.0s finished
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fe88c2a1520>


      ...replicating address
         ...Base
         ...Map street suffixes...
         ...Mapped & split by street suffixes...
         ...Number processing...

         Address dataframe expansion is complete! (n=900)
         ...Base
         ...Number processing...
         House number dataframe expansion is complete! (n=900)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/core/arrays/_arrow_utils.py:9: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _pyarrow_version_ge_015 = LooseVersion(pyarrow.__version__) >= LooseVersion("0.15")


      ...merge user input & lookup table
      ...mapping


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
100%|██████████| 1/1 [00:04<00:00,  4.83s/it]

Directory already exists
Output saved
   [Completed] Mapping geo data

[Completed] Preparing geo data

[Start] Preparing ACS data
   [Start] Validating ACS input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing, this may impact the ability to return race approximations



/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)



   [Completed] Validating ACS input data

   ...loading ACS lookup tables
   ... combining ACS & user input data
 ...Copy dataframes
 ...Block group
 ...Census tract
 ...Zip code
 ...No match
 ...Merge
 ...Merging complete
[Complete] Preparing ACS data

CPU times: user 34.4 s, sys: 10.7 s, total: 45.1 s
Wall time: 22.7 s


### Invoke the ZRP_Predict on the sample data
To generate predictions, you can:
1. Provide the path to the preferred pipeline directory in the `__init__`. 
    - Here we provide the default path

In [8]:
curpath = os.getcwd()
pipe_path = join(curpath, "../../zrp/modeling/models")

In [9]:
zrp_predict = ZRP_Predict(pipe_path)

To transform the data/generate predictions, provide the prepared data from ZRP_Prepare to the transform

In [11]:
zrp_predict.fit(prepared)
zrp_output = zrp_predict.transform(prepared)

   [Start] Validating pipeline input data
     Number of observations: 1425
     Is key unique: False
       (Warning!!) middle_name is 67.15789473684211% missing, this may impact the ability to return race approximations
   [Completed] Validating pipeline input data



100%|██████████| 1/1 [00:00<00:00, 932.27it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1/1 [00:00<00:00, 1116.10it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1/1 [00:00<00:00, 1274.86it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


Directory already exists
Output saved


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


### Inspect the output
- Preview the data
- View what artifacts are saved

In [18]:
zrp_output.sort_values("source_block_group", ascending=False)

,AAPI,AIAN,BLACK,HISPANIC,WHITE,race_proxy,source_block_group,source_census_tract,source_zip_code,source_bisg
ZEST_KEY,,,,,,,,,,
138,0.002625,0.001921,0.480969,0.016531,0.497954,WHITE,1.0,0.0,0.0,0.0
322,0.017464,0.000595,0.002867,0.120008,0.859067,WHITE,1.0,0.0,0.0,0.0
320,0.080177,0.011591,0.024196,0.051094,0.832942,WHITE,1.0,0.0,0.0,0.0
319,0.001641,0.001610,0.021942,0.058487,0.916320,WHITE,1.0,0.0,0.0,0.0
318,0.027923,0.000558,0.222414,0.042709,0.706395,WHITE,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
435,0.021511,0.000899,0.009937,0.043853,0.923800,WHITE,0.0,0.0,1.0,0.0
434,0.000482,0.000465,0.001865,0.001348,0.995840,WHITE,0.0,0.0,1.0,0.0
43,0.161509,0.000540,0.001793,0.002106,0.834052,WHITE,0.0,0.0,1.0,0.0


### Check Coverage
A quick glance at the ZRP output we can see a low missing rate. `ZRP_Predict` uses a waterfall method that predicts by using block group, census_tract, then zip_code. 

In [23]:
zrp_output.filter(regex='[A-Z]|race').isna().mean()

AAPI          0.017316
AIAN          0.017316
BLACK         0.017316
HISPANIC      0.017316
WHITE         0.017316
race_proxy    0.017316
dtype: float64

Checking the distribution of predicted race & ethnicity 

In [20]:
zrp_output.race_proxy.value_counts(normalize=True, dropna=False)

WHITE       0.876623
BLACK       0.047619
HISPANIC    0.034632
AAPI        0.023810
NaN         0.017316
Name: race_proxy, dtype: float64

In [25]:
zrp_output.shape

(462, 10)

Please refer to the source columns to determine which geographic identifier or method was used to generate the proxy 

`ZRP_Predict` generates multiple artifacts that are automatically saved:
- Dataframe with proxies
    - `proxy_output.feather`
- Validation dictionary for input data
    - `input_predict_validator.json`


In [14]:
!ls artifacts

input_acs_validator.json      proxy_output.feather
input_predict_validator.json  Zest_Geocoded_test_2019__34.parquet
input_validator.json
